In [11]:
## library
import random
import numpy as np
import torch
from torch.autograd.variable import Variable
from torch import nn, optim
import argparse
import visdom
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import sys
import matplotlib.pyplot as plt
import gan_pytorch
import numpy as np
from keras.utils.vis_utils import plot_model
import time
from tqdm import tqdm
import tensorflow as tf
from torchsummary import summary
from torch.utils.data import Subset



##Classifier pytorch    

In [12]:
noise_dim=100
batch_size=64
num_img_channel=1
img_size=28
sample_interval=64          



In [3]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()

        def convlayer(n_input,n_output,k_size=4,stride=2, padding=2, normalize=True,dilatation=1):
            block=[nn.Conv2d(n_input,n_output,kernel_size=k_size,stride=stride,padding=padding,bias =False,dilation=dilatation),]

            if normalize:
                block.append(nn.BatchNorm2d(n_output))
            block.append(nn.LeakyReLU(0.18,inplace=True))
            return block
        
        self.conv_block=nn.Sequential(
            *convlayer(num_img_channel, 32,5,2,2,normalize=True),   
            *convlayer(32,64,5,2,2),
        )
        self.fc_block= nn.Sequential(
            nn.Linear(64*7*7,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.18,inplace=True),
            nn.Linear(512,10),
        )

    def forward(self,img):
        conv_out=self.conv_block(img)
        conv_out=conv_out.view(img.size(0),64*7*7)
        l2_value = self.fc_block(conv_out)
        l2_value = l2_value.unsqueeze_(dim=2).unsqueeze_(dim=3)
        return l2_value

In [13]:

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)



In [14]:
#Dataset Mnist:
def mnist_data():
    compose = transforms.Compose(
        [transforms.Resize(img_size),
         transforms.ToTensor(),
         transforms.Normalize((0.5,), (0.5,)),  # Remarquez que les paramètres sont des tuples à un élément seulement
         transforms.Grayscale()
         ])
    output_dir = './data/mnist'
    return datasets.MNIST(root=output_dir, train=True,
                          transform=compose, download=True)


mnist = mnist_data()
train_indices = range(0, int(0.8 * len(mnist)))  # Utilisez les 80% premières images pour l'entraînement
train_dataset = Subset(mnist, train_indices)



batch_iterator = DataLoader(train_dataset, shuffle=True, batch_size=batch_size) # List, NCHW format.


cuda = torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor



In [15]:

classifier=Classifier()
classifier=classifier.cuda()

classifier



Classifier(
  (conv_block): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.18, inplace=True)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.18, inplace=True)
  )
  (fc_block): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.18, inplace=True)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [16]:
display_server="http://localhost"
display_port=8097
try:
    
    vis = visdom.Visdom(server=display_server, port=display_port, raise_exceptions=True) # Create vis env.
except ImportError:
    vis = None
else:
    vis.close(None) # Clear all figures.



Setting up a new session...


In [17]:
lr=0.0001
b1=0.5
b2=0.999
optimizer=optim.Adam(classifier.parameters(),lr=lr,betas=(b1,b2))
loss_C=nn.CrossEntropyLoss()
seed = Variable(Tensor(25, noise_dim, 1, 1).normal_(0, 1), requires_grad=False) # To track the progress of the GAN.


In [18]:
from tqdm import tqdm

def train(n_epochs):
    for epoch in tqdm(range(n_epochs)):
        print('Epoch {}'.format(epoch))
        for i, (batch, labels) in enumerate(batch_iterator):
            
            imgs = Variable(batch.type(Tensor), requires_grad=False)
            
        
            labels = Variable(labels.type(Tensor), requires_grad=False)
            optimizer.zero_grad()

            output = classifier(imgs)

            loss = loss_C(output, labels.unsqueeze(1).unsqueeze(2).long() )

            loss.backward()
            optimizer.step()


In [19]:
train(3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 0


 33%|███▎      | 1/3 [00:17<00:34, 17.47s/it]

Epoch 1


 67%|██████▋   | 2/3 [00:29<00:14, 14.36s/it]

Epoch 2


100%|██████████| 3/3 [00:41<00:00, 13.92s/it]


In [20]:
test_indices = range(int(0.8 * len(mnist)), len(mnist))
test_dataset = Subset(mnist, test_indices)



In [21]:
K=random.randint(1,12000)
vis.images(mnist.data[48000+K,:,:])
images=mnist.data[test_indices,:,:].unsqueeze(1).cuda().float()
classifier(images)[K,:,:,:].argmax()





tensor(0, device='cuda:0')

In [23]:
labels=mnist.targets[48000:].unsqueeze(1).unsqueeze(2).cuda()   
predicted=torch.argmax(classifier(images),dim=1)
(torch.sum(labels==predicted)*100/12000)





tensor(98.4000, device='cuda:0')